# RUL prediction based on synthetic training data
In this notebook we will perform RUL prediction on CMAPSS data, but instead of using the original data we use a synthetic variant. The generation of synthetic data is performed in [this notebook](). The baseline and some introduction on RUL prediction is found in [this notebook]().

In [1]:
# import various modules and packages
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt


In [2]:
# load the synthetic data as training data
train = pd.read_csv('./CMAPSS/Synthetic/DeepEcho_FD001_RUL.csv')
train = train.drop('Unnamed: 0', axis=1)

# include all columns apart from RUL
col_names = train.columns[:-1]

# read data
test_data = pd.read_csv(('./CMAPSS/test_FD001.txt'), sep='\s+', header=None, names=col_names)
test_labels = pd.read_csv(('./CMAPSS/RUL_FD001.txt'), sep='\s+', header=None, names=['RUL'])

# show synthetic data
test_data

,unit_nr,ops_set1,ops_set2,ops_set3,sens_1,sens_2,sens_3,sens_4,sens_5,sens_6,...,sens_12,sens_13,sens_14,sens_15,sens_16,sens_17,sens_18,sens_19,sens_20,sens_21
1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,21.61,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,21.61,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,21.61,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,21.61,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,21.61,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,21.61,...,520.69,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974
100,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,21.61,...,521.05,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771
100,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,21.61,...,521.18,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051
100,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,21.61,...,521.33,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699


In [3]:
# Compute Remaining Useful Life (RUL) for each index (engine)
def add_remaining_useful_life(df):
    # Get the total number of cycles for each unit
    grouped_by_unit = df.groupby(by="unit_nr")
    max_cycle = grouped_by_unit["timecycle"].max()
    
    # Merge the max cycle back into the original frame
    result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='unit_nr', right_index=True)
    
    # Calculate remaining useful life for each row
    remaining_useful_life = result_frame["max_cycle"] - result_frame["timecycle"]
    result_frame["RUL"] = remaining_useful_life
    
    # drop max_cycle as it's no longer needed
    result_frame = result_frame.drop("max_cycle", axis=1)
    return result_frame

# train = add_remaining_useful_life(train)

In [4]:
# Preprocess the data, remove constant columns, indexes and operational settings.
print("Train set after removal of constant values:")
const_cols = ['sens_1', 'sens_5', 'sens_6', 'sens_10', 'sens_16', 'sens_18', 'sens_19']

op_settings = ["ops_set1", "ops_set2", "ops_set3"]

cols_to_drop = ["unit_nr"] + op_settings + const_cols

train = train.drop(cols_to_drop, axis=1)
train

Train set after removal of constant values:


,sens_2,sens_3,sens_4,sens_7,sens_8,sens_9,sens_11,sens_12,sens_13,sens_14,sens_15,sens_17,sens_20,sens_21,RUL
0,642.473068,1590.523119,1411.264748,553.620924,2387.983560,9066.693171,47.541168,521.428812,2388.100523,8173.048289,8.442146,392.952618,38.901845,23.289705,84.186201
1,642.901593,1592.297732,1399.935190,553.624768,2388.133781,9078.790032,47.541168,521.411841,2388.096152,8153.491199,8.455641,392.893241,39.019725,23.270599,101.649576
2,642.488421,1589.557587,1413.170862,553.297483,2388.110799,9069.687832,47.839807,520.937211,2388.112223,8140.691979,8.471263,392.541833,38.862521,23.289705,93.164509
3,642.377192,1586.122690,1411.388159,553.507911,2388.157503,9088.595165,47.421999,521.274816,2388.117313,8145.023711,8.394715,393.828368,38.730699,23.233689,96.976067
4,642.695678,1591.227074,1410.420718,553.427153,2388.084332,9064.356090,47.414871,521.698415,2388.100825,8130.063203,8.435684,393.670542,38.756972,23.335115,91.736228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25721,642.401271,1593.685517,1413.238487,553.279646,2388.125408,9067.512259,47.523497,521.586056,2388.122424,8138.288461,8.462023,394.808076,38.952301,23.291737,124.881773
25722,642.571931,1589.853140,1412.539083,553.286837,2388.087885,9068.586140,47.495900,521.148832,2388.096152,8151.826193,8.431912,392.546345,38.911342,23.273871,109.955212
25723,642.353279,1588.698769,1410.207921,552.365619,2388.107631,9081.744072,47.751174,520.750551,2388.083559,8126.134846,8.433762,392.104497,38.825385,23.338452,126.050395
25724,642.819208,1587.417323,1408.486266,553.411608,2388.105514,9083.899100,47.633874,521.486042,2388.103948,8155.002869,8.434369,393.644056,38.757235,23.357953,107.807862


# Prediction!

In [5]:
def evaluate(y_true, y_hat, label='test'):
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    variance = r2_score(y_true, y_hat)
    print('{} set RMSE:{}, R2:{}'.format(label, rmse, variance))

In [6]:
# remove RUL from training data, split
train_data = train.drop('RUL', axis=1) # data without constants
train_labels = train.pop('RUL') # RUL values

# Since the true RUL values for the test set are only provided for the last time cycle of each engine, 
# the test set is subsetted to represent the same
test_data = test_data.groupby('unit_nr').last().reset_index().drop(cols_to_drop, axis=1)

# # check whether RUL and constant columns removed
# X_test

In [7]:
rf = RandomForestRegressor()

rf.fit(train_data, train_labels)

# predict and evaluate
pred_train = rf.predict(train_data)
evaluate(pred_train, train_labels, 'train')

pred_test = rf.predict(test_data)
evaluate(pred_test, test_labels)

train set RMSE:9.913156983258949, R2:0.6496482594793125


ValueError: Found input variables with inconsistent numbers of samples: [303, 100]

# Discussion
First of all, we see that the RMSE is way higher than expected. As we were struggling at first with finding a synthetic data generator that can correctly model timeseries, we expect this may be a cause. To check this hypothesis, we will plot the difference sensors compared to RUL.

In [ ]:
# reload synthetic data
train = pd.read_csv('./CMAPSS/Synthetic/SDV_FD001_RUL.csv')
train = train.drop('Unnamed: 0', axis=1)
train = add_remaining_useful_life(train)

# create list with sensors
sensors = []
for i in range(1,22):
    sensors.append(f"sens_{i}")

# create plot
fig, axs = plt.subplots(nrows=7, ncols=3, figsize=(15, 24))

for i, sensor_idx in enumerate(sensors):
    row = i // 3
    col = i % 3
    ax = axs[row, col]
    # plot each sensor in own graph
    for j in train['unit_nr'].unique():
        ax.plot('RUL', sensor_idx, data=train[train['unit_nr']==j])
    
    # graphics
    ax.set_xlim(250, 0)
    ax.set_xticks(np.arange(0, 275, 25))
    ax.set_ylabel(sensor_idx)
    ax.set_xlabel(f'RUL set out to value of {sensor_idx}.')

plt.tight_layout()
plt.savefig('./Figures/Obtained')

In [ ]:
# create plot
fig, axs = plt.subplots(nrows=7, ncols=3, figsize=(15, 24))

# Compute Remaining Useful Life (RUL) for each index (engine)
def add_remaining_useful_life(df):
    # Get the total number of cycles for each unit
    grouped_by_unit = df.groupby(by="unit_nr")
    max_cycle = grouped_by_unit["timecycle"].max()
    
    # Merge the max cycle back into the original frame
    result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='unit_nr', right_index=True)
    
    # Calculate remaining useful life for each row
    remaining_useful_life = result_frame["max_cycle"] - result_frame["timecycle"]
    result_frame["RUL"] = remaining_useful_life
    
    # drop max_cycle as it's no longer needed
    result_frame = result_frame.drop("max_cycle", axis=1)
    return result_frame

syn_data = train
data = pd.read_csv(('./CMAPSS/train_FD001.txt'), sep='\s+', header=None, names=col_names)
data = add_remaining_useful_life(data)

for i, sensor_idx in enumerate(sensors):
    row = i // 3
    col = i % 3
    ax = axs[row, col]
    # plot each sensor in own graph
    ax.plot('RUL', sensor_idx, data=syn_data[syn_data['unit_nr']==297], label='Synthetic')
    ax.plot('RUL', sensor_idx, data=data[data['unit_nr']==1], label='Original')
    
    # graphics
    ax.set_xlim(250, 0)
    ax.set_xticks(np.arange(0, 275, 25))
    ax.set_ylabel(sensor_idx)
    ax.set_xlabel(f'RUL set out to value of {sensor_idx}.')

plt.legend()
# plt.show()
plt.savefig('./Figures/Syn-vs-Original')